# Learning the parameters of a MRSort model using the metaheuristic algorithm (oso-pymcda)

## Introduction

The metaheuristic code at our disposal comes from github (https://github.com/oso/pymcda) developped by Olivier Sobrie). In addition to this code, we took into account mainly these adaptations : the learning a MRSort model from "a duplicated data set", the generation of statistic plots on the learning results. The code originally in Python 2 has been upgraded to some extent to Python 3. This code is located in the "oso-pymcda/" directory, which is in the same directory as this notebook.

## Settings

Before digging into the code, here are some requirements to have : 
   * The library matplotlib.pyplot need to be installed. This can be done with the command line below (preferably using pip  - that can be also installed following the instructions of this link : https://pip.pypa.io/en/stable/installing/):

In [1]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


   * Download and install CPLEX Optimization Studio : https://www.ibm.com/products/ilog-cplex-optimization-studio (choose the student/teacher free edition) and follow the steps. Help could be found here : https://www.ibm.com/support/knowledgecenter/SSSA5P_12.9.0/ilog.odms.studio.help/Optimization_Studio/topics/COS_home.html
   

We need to set the global variable *DATADIR* so that it contains the right path from the root to this working directory  **MRSort-jupyter-notebook** . Here an example :

In [2]:
%env DATADIR /Users/pegdwendeminoungou/python_workspace/MRSort-jupyter-notebook

env: DATADIR=/Users/pegdwendeminoungou/python_workspace/MRSort-jupyter-notebook


## Some helpful articles on MCDA/MRSort model

In order to grasp the methodology of MCDA and have a overview of the metaheuristic that is used in this notebook, we give here below some useful related articles :
   * [A new decision support model for preanesthetic evaluation](papers/Sobrie_and_al.pdf)
   * [Learning monotone preferences using a majority rule sorting model](papers/Sobrie_Mousseau_Pirlot.pdf) (in particular, this explains with details the procedure of the metaheuristic)
   * [Learning the Parameters of a Multiple Criteria Sorting Method Based on a Majority Rule](papers/Leroy_Mousseau_Pirlot.pdf)

## Description of the code

The code gathers mainly 2 parts : 
   * the first component on the generation, learning and tests of a parametered MRSort model (one running of the learning algorithm followed by tests),
   * the second component on the compilation of series of parametered MRSort runnings and the output of interesting statistic plots.

### The first component

First, we need to move to our working environment and run the main file in order to keep in memory the implementations of functions :

In [4]:
run oso-pymcda/apps/random_model_generation_msjp.py

Second, to achieve our goal we follow these steps : 
   * <u>Step 1</u> : initialize the required parameters,
   * <u>Step 2</u> : generate a new random MRSort model (profile, weights, threshold),
   * <u>Step 3</u> : generate randomly a set of alternatives and performance table,
   * <u>Step 4</u> : assign categories to these alternatives to yield a learning set,
   * <u>Step 5</u> : run the MRSort metaheuristic learning algorithm,
   * <u>Step 6</u> : validate the learning of the random model (% of classification "initial model VS learned model" on the learning set)
   * <u>Step 7</u> : test the learned algorithm on a benchmarch of alternatives examples
   * <u>Step 8</u> : show the important results (summarized also in a csv file)
   
   

#### Step 1 : initialize the required parameters

Here, we initialize the parameters of one running for the learning algorithm. Respectively, we have : 
   * *nb_categories* : the number of categories (classes)
   * *nb_criteria* : the number of criteria taken in consideration of the MCDA problem
   * *nb_alternatives* : the number of alternatives (for the learning set)
   * *dir_criteria* : the list of order/direction on preferences of the criteria  (1 for a criteria to maximize)
   * *l_dupl_criteria* : the list of criteria (indices) to duplicate during the learning process
   * *nb_tests* : the number of tests (number of alternatives) to carry out in order to compare the performance of the learned model regarding the initial model
   * *nb_models* : the number of models that independantly learn during one running of the learning algorithm
   * *meta_l* : the number of iteration of the metaheuristic algorithm (outer loop)
   * *meta_ll* : the number of iteration of the metaheuristic algorithm (inner loop)
   * *meta_nb_models* : the number of models (population) handled by the metaheuristic (evolutionary) algorithm during the learning process

In [5]:
nb_categories = 2 # fixed
nb_criteria = 5
nb_alternatives = 100
dir_criteria = [1]*nb_criteria # fixed to 1 for all criteria
l_dupl_criteria = list(range(nb_criteria))[:1]

# parameters of test
nb_tests = 10000
nb_models = 10

# parameters of the metaheuristic MRSort
meta_l = 10
meta_ll = 10
meta_nb_models = 10

Now we can create an instance of the one running of the learning algorithm as follows :

In [6]:
inst = RandMRSortLearning(nb_alternatives, nb_categories, 
        nb_criteria, dir_criteria, l_dupl_criteria, 
        nb_tests, nb_models, meta_l, meta_ll, meta_nb_models)

#### Step 2 to 4 : generate a new random MRSort model, alternatives and assignments

Here 3 steps are performs one after the other in the same function. We generate a new random MRSort, then we generate alternatives, and finally we assign these alternatives in 2 categories regarding the MRSort rule of the given model. In addition to these 3 operations, we introduce a coefficient that enable us to control the balance between the set of alternatives (number of alternatives) sorted in the categories.

In [7]:
inst.generate_random_instance()

We can have a look on the model that have been generated :
   * generated parameters of the model MRSort

In [8]:
inst.model.bpt.display() # display the limit profile of the random model b1

      c1    c2    c3    c4    c5 
b1 0.927 0.433 0.082 0.071 0.137 


In [9]:
inst.model.cv.display() # display the weights of each criteria of the model w

     c1    c2    c3    c4    c5 
w 0.157 0.338 0.052 0.144 0.308 


In [10]:
print("lambda\t%.7s" % inst.model.lbda) 

lambda	0.633


   * performance table of generated alternatives

In [11]:
inst.pt.display()

        c1    c2    c3    c4    c5 
a1   0.616 0.913 0.789 0.983  0.64 
a10  0.282 0.506 0.517 0.915 0.012 
a100  0.12 0.636 0.774 0.312 0.234 
a11  0.312 0.644 0.319  0.99 0.489 
a12  0.558 0.062 0.026  0.24 0.518 
a13  0.804 0.224 0.834 0.352 0.739 
a14  0.404 0.624 0.465 0.448 0.753 
a15  0.897 0.471  0.38 0.983 0.722 
a16   0.39 0.131  0.98 0.442 0.984 
a17  0.544 0.788  0.38  0.37  0.67 
a18  0.785 0.339 0.068 0.396 0.332 
a19  0.805 0.017   0.3 0.387 0.789 
a2   0.504 0.191 0.616 0.624  0.03 
a20  0.457 0.944 0.286 0.664 0.362 
a21  0.209 0.846 0.349 0.624 0.385 
a22  0.079 0.712 0.382 0.915 0.731 
a23  0.094 0.662 0.237 0.301 0.793 
a24  0.691 0.469 0.209 0.789 0.642 
a25   0.91 0.389 0.802 0.722  0.34 
a26  0.213 0.348 0.351 0.498 0.567 
a27  0.186 0.856  0.93 0.301 0.151 
a28  0.005  0.62 0.249 0.032 0.437 
a29  0.664  0.87 0.245 0.412 0.053 
a3    0.04 0.585 0.525 0.312 0.859 
a30  0.496 0.687 0.743 0.902 0.266 
a31  0.845 0.326 0.552 0.801 0.987 
a32  0.983 0.193 0.694 0.932

   * the result of the assignment of alternatives

In [12]:
inst.aa.display()

     category
a1       cat1
a10      cat2
a100     cat1
a11      cat1
a12      cat2
a13      cat2
a14      cat1
a15      cat1
a16      cat2
a17      cat1
a18      cat2
a19      cat2
a2       cat2
a20      cat1
a21      cat1
a22      cat1
a23      cat1
a24      cat1
a25      cat2
a26      cat2
a27      cat1
a28      cat1
a29      cat2
a3       cat1
a30      cat1
a31      cat2
a32      cat1
a33      cat1
a34      cat1
a35      cat2
a36      cat1
a37      cat1
a38      cat2
a39      cat2
a4       cat2
a40      cat2
a41      cat1
a42      cat2
a43      cat1
a44      cat1
a45      cat2
a46      cat2
a47      cat1
a48      cat1
a49      cat2
a5       cat2
a50      cat1
a51      cat2
a52      cat2
a53      cat2
a54      cat2
a55      cat2
a56      cat2
a57      cat2
a58      cat1
a59      cat1
a6       cat1
a60      cat1
a61      cat1
a62      cat1
a63      cat1
a64      cat2
a65      cat1
a66      cat1
a67      cat1
a68      cat2
a69      cat1
a7       cat2
a70      cat2
a71      cat2
a72   

#### Step 5: run the MRSort metaheuristic learning algorithm

This following step represents one model iteration of the metaheuristic algorithm. This iteration learns with a single model the initial model from the previous learning set (performance table and assignments of alternatives).

In [13]:
inst.num_model = 0 # initialization of the position of the model that is currently learning
execution_time = inst.run_mrsort()
print("Time (s) : %f" % execution_time) # computational time of the running

Time (s) : 0.550408


We show the parameters of the model learned :

In [14]:
inst.model2.bpt.display()
inst.model2.cv.display()
print("lambda\t%.7s" % inst.model2.lbda) 

        c1     c1d      c2      c3    c4    c5 
b1 0.91301 0.02899 0.42801 0.90201 0.175 0.217 
      c1    c1d     c2     c3    c4     c5 
w 0.9994 0.0001 0.0003 0.0001     0 0.0001 
lambda	0.00039


#### Step 6 : validate the learning of the random model

We can calculate the rate of validation of the model (which is the percentage for the learned model to find the good classifications compared to assignments given by the original model) regarding the learning set.

In [15]:
ca_v,cag_v = inst.eval_model_validation() # calculating the validation rate
print("validation rate : %f" % ca_v)

validation rate : 1.000000


We can also draw the confusion matrix of the validation phase :

In [16]:
matrix = compute_confusion_matrix(inst.aa, inst.aa_learned, inst.model.categories) # construction of the confusion matrix
print_confusion_matrix(matrix, inst.model.categories) # printing the confusion matrix

     cat2 cat1 
cat2   49    0 
cat1    0   51 


#### Step 7 : test the learned algorithm on a benchmarch of alternatives examples

Analogously, we can calculate the test rate (which is the percentage for the learned model to find the good classification compared to right assignments given by the original model) regarding a test set.

In [17]:
ao_tests,al_tests,ca_t,cag_t = inst.eval_model_test()
print("test rate : %f" % ca_t)

test rate : 0.941000


#### Step 8 : show the important results

In order to show the final results, we need to achieve all the tests ; in fact, until now we only compute one learned model. Therefore, it is important to carry out the runnings and yield *nb_models* learned models. To do so, we can straightforwardly execute :  

In [20]:
inst.run_mrsort_all_models()

In [21]:
DATADIR

'/Users/pegdwendeminoungou/python_workspace/MRSort-jupyter-notebook'

As a result, all the tests are done and we have also generated a csv file summarizing the tests and giving details of each one. This file is found on the directory *rand_valid_test_na100_nca2_ncr5-0_dupl1* visible from the root directory of this notebook. The file name begins with "valid_test_dupl...." .

Another csv file is the file that contains more compact data facilitating the drawing of different plots. This file is generated with the command line :

In [22]:
inst.report_plot_results_csv()

It yields a csv file, which name begins with "plot_results...." in the same directory as the previous file.

The final function of this section is the function that ouputs a instance of the learning algorithm (criteria, categories, performance tables and assignments, all codified in a customized syntax)

In [23]:
inst.build_osomcda_instance_random()

'/Users/pegdwendeminoungou/python_workspace/MRSort-jupyter-notebook/rand_valid_test_na100_nca2_ncr5-0_dupl1//osomcda_rand-100-2-5-1-20191029-115742.csv'

This output file is also in the previous directory as the previous files.

### The second component

In this section, we will use what have been done in the previous section as a unit test, and then we will repeat it several times, varying different parameters.

We give here the call of a unit test : 

In [24]:
inst.learning_process()

In order to read in advance the implementation of the functions of this part, we run :

In [26]:
run oso-pymcda/apps/learning_random_models_results.py

At the beginning of the series of tests, some parameters must be set:

In [27]:
nb_categories = 2 #fixed
nb_criteria = 6

ticks_criteria = list(range(0,nb_criteria+1,2)) # ticks on plots results representing the number fo criteria
ticks_alternatives = list(range(50,200,50)) # ticks on plots results representing the number of alternatives

nb_tests = 10000
nb_models = 10

#Parameters of the metaheuristic MRSort
meta_l = 10
meta_ll = 10
meta_nb_models = 10
directory = DATADIR
output_dir = DATADIR + "/learning_results_plots"

Four variables have not been explained yet. These are :
   * *ticks_criteria* : the range of the ticks values corresponding to the number of duplicated criteria. One tick corresponds to a unit test made on a given number of duplicated criteria.
   * *ticks_alternatives* : the range of the ticks values of the number of alternatives taken into consideration in the multiple test process. One tick represents a number of alternatives taken into account on a unit test.

We can apply now a function that will compute series of unit tests according to the range of values of the parameters given.

We defined a function (***exec_all_tests***) to execute this bunch of tests with different experiment protocols. It runs each unit test with a different random model and different set of alternatives.

So, let's construct an instance of that sort and then run ***exec_all_tests*** :

In [28]:
tests_instance = MRSortLearningResults(directory, output_dir, nb_categories, nb_criteria,ticks_criteria,ticks_alternatives, \
                nb_tests, nb_models, meta_l, meta_ll, meta_nb_models)

In [29]:
tests_instance.exec_all_tests()

 ... unit test nb_alternatives = 50, nb_duplicated_criteria = 0
 ... unit test nb_alternatives = 50, nb_duplicated_criteria = 2
 ... unit test nb_alternatives = 50, nb_duplicated_criteria = 4
 ... unit test nb_alternatives = 50, nb_duplicated_criteria = 6
 ... unit test nb_alternatives = 100, nb_duplicated_criteria = 0
 ... unit test nb_alternatives = 100, nb_duplicated_criteria = 2
 ... unit test nb_alternatives = 100, nb_duplicated_criteria = 4
 ... unit test nb_alternatives = 100, nb_duplicated_criteria = 6
 ... unit test nb_alternatives = 150, nb_duplicated_criteria = 0
 ... unit test nb_alternatives = 150, nb_duplicated_criteria = 2
 ... unit test nb_alternatives = 150, nb_duplicated_criteria = 4
 ... unit test nb_alternatives = 150, nb_duplicated_criteria = 6


The results are compiled into several folders beginning by "rand_valid_test...". Each of them contains the result of a single unit test.

Finally and after having the results, we can run the program that shows the graphical representations.

In [30]:
tests_instance.plot_all_results()

The output is a folder named "learning_results_plots" containing different comparative plots.

We have four types of plots subdivized as follows:
   * *computational_time* : in this folder, we draw the evolution of the execution time of a running of the learning process depending on 2 factors : the number of alternatives and the number of duplicated criteria (those criteria we want to learn the preference direction from). Following the first factor, we have individual plots with fixed number of duplicated criteria (time_nb_alternatives_dupl...), as well as a summary plot (time_nb_alternatives_dupl_all). Following the second factor, we also have individual plots with the number of alternatives fixed (time_dupl_crit_na...), as well as a summary plot (time_dupl_crit_na_all).
   * *validation_CA* : these plots are about the percentage of restoration of the learning set by the learned model compared to the original model. The arrangement of the plots in 2 factors is the same as for *computational_time* (the first point).
   * *tests_CA* : these plots concern the percentage of restoration of the learning test by the learned model compared to the original model.The arrangement of the plots in 2 factors is the same as for *computational_time* (the first point).
   * *restitution* : these plots concern the percentage of the ability to deduce the preference directions of the duplicated criteria. [For example, considering one criteria and its duplicate, we assume that the preference direction is found if in a case of maximization of this criteria, its duplicate (which is a criteria that incite to minimize) is found to be equal to 0. In fact, this duplicate isn't taken into account by the model.] The arrangement of the plots in 2 factors is the same as for *computational_time* (the first point).